## THESIS: MSc Economics
*Quantitative Economics, EC 475* <br>
*The London School of Economics and Political Science* <br>
*Thesis Advisor: Prof. Xavier Jaravel*

In [30]:
import scipy
import statsmodels.api as sm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from statsmodels.iolib.summary2 import summary_col
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")
plt.rcParams['axes.facecolor'] = 'white'

## Leave One Out Test

In [36]:
def p_val(loss_1, loss_2, RSS_accumulator, algo1_result, algo2_result):
    # Printing the RSS accumulator,
    #print(RSS_accumulator)

    # Now, the following is the algorithm we need to implement: 
    # I drop the chosen model
    # I find the difference between the RSS for original model and all the other models.
    # This gives me the loss suffered due to not choosing those other models
    # Find the percentage of such losses that are more extreme than the chosen model;
    # This gives me the p-value of the model

    #print(algo1_result)
    #print(algo2_result)
    count1 = 0
    count2 = 0

    losses_1 = {}
    losses_2 = {}
    for i in RSS_accumulator: 
        if i != algo1_result:
            losses_1[i] = RSS_accumulator[i] - RSS_accumulator['Original']
            if losses_1[i] > loss_1:
                count1 += 1
        if i != algo2_result:
            losses_2[i] = RSS_accumulator[i] - RSS_accumulator['Original']
            if losses_2[i] > loss_2:
                count2 += 1
    # returning the p-values
    return (count1/4, count2/4)

## Data Generation Code

In [37]:
# setting the number of observations; eventually, make this a loop
def synthetic_data_generator(n = 100):

    # Not setting random seed by design; we would like to keep the regressors stochastic, they might change their
    # performance across the state space.

    # rather than drawing from a uniformly random distribution, I uniformly randomly choose a distribution
    # out of several different distributions and then generate regressors based on these 

    # all the regressors are either chosen at random from a uniform distribution or a Gaussian distribution; 

    coefficients = np.random.default_rng().normal(loc = np.random.randint(low = 0, high = 20, size = 1), scale = 20, size = 4)/10
    choice_of_distribution = np.random.randint(2)
    if choice_of_distribution == 0:
        #print('Choosing data uniformly at random.. ')
        data = np.random.rand(n, 4)
    elif choice_of_distribution == 1:
        #print('Choosing data from a Gaussian Distribution.. ')
        data = np.random.default_rng().normal(loc = 0, scale = 1, size = (n, 4))

    coefficients = np.transpose(coefficients)
    data = pd.DataFrame(data, columns = ['x1', 'x2', 'x3', 'x4'])

    # intermediate step for creating the outcome variable

    intermediate = data.copy(deep = True)
    intermediate['x1_square'] = intermediate['x1']*intermediate['x1']
    intermediate['x2'] = np.log(intermediate['x2'])
    intermediate['cross'] = intermediate['x2']*intermediate['x3']
    intermediate = intermediate.drop(columns = ['x2', 'x3'])

    # changing the columns of the pandas dataframe to what I want in the true model for the outcome variable
    intermediate = np.matmul(intermediate, coefficients)

    # adding error to the outcome I created 
    intermediate = pd.DataFrame(np.random.default_rng().normal(loc = 0, scale = 1, size = (n, )) + intermediate, columns = ['y'])
    #intermediate = pd.DataFrame(intermediate, columns = ['y'])
    data.insert(0, 'y', intermediate)
    data = data.dropna()

    #print(data)
    del intermediate 
    return data

## Starting Exercise: MinMax Principle

In [38]:
# defining the minimiax function
# this function takes as input the residual sums of squares (the solution to this stylised optimisation problem) 
# and returns the solution that is chosen by the minimax principle
def miniMax(RSS_accumulator):
    return max(RSS_accumulator, key = RSS_accumulator.get)

## Main Exercise: MiniMax Regret
The main focus of this algorithm is to choose that model which has the minimum maximum loss. In our settings, the loss is defined by the difference between the minimum RSS between the incorrect model and the correct model.

In [39]:
# define the minimax regret function; functionally speaking, find the regret first, then span the space
# of the regressors and get the model with the least regret across all models.

# if model A has RSS n1 and model B has RSS n2, where model B is the more correct representation of the world,
# the regret, if model A is chosen is given by: n1 - n2 (the system suffers a loss of these many RSS units)
def regret(n1, n2):
    return n1 - n2

def minimax_regret(RSS_accumulator):
    max_regrets = list()
    for i in range(1, len(RSS_accumulator) + 1):
        temp = list()
        for j in range(1, len(RSS_accumulator) + 1):
            if i != j:
                temp.append(regret(RSS_accumulator['result ' + str(i)], RSS_accumulator['result ' + str(j)]))
        max_regrets.append(max(temp))
    #print(max_regrets)
    return 'result ' + str(min(range(len(max_regrets)), key=max_regrets.__getitem__) + 1)

 _In_ this section of the code, we start claiming a model to be 
 a 'true' model, offer a variety of alternatives that might seem plausible based on economic theory
 and see which one of these models is chosen by our different Decision theoretic algorithms.
 Let the true model in this case be: <br>
 $y_i = \alpha + \beta_1 * x_{1i} + \beta_2 * ln(x_{1i}^2) + \beta_3 * ln(x_{2i})*x_{3i} + \beta_4 * x_{4i} + \varepsilon_i$

In [40]:
def combinator(x_val = 3000):
    loss_1 = list()
    loss_2 = list()
    index = 0
    p_1 = 0
    p_2 = 0
    for i in range(100, x_val, 100):
        data = synthetic_data_generator()
        outcome = data['y']
        RSS_accumulator = dict()

        intermediate = data.copy(deep = True)
        intermediate['x1_square'] = intermediate['x1']*intermediate['x1']
        intermediate['x2'] = np.log(intermediate['x2'])
        intermediate['cross'] = intermediate['x2']*intermediate['x3']
        intermediate = intermediate.drop(columns = ['x2', 'x3'])
        intermediate = sm.add_constant(intermediate)
        intermediate = intermediate.drop(columns = ['y'])
        original = sm.OLS(outcome, intermediate).fit()
        RSS_accumulator['Original'] = original.ssr


        # HYPOTHESISED MODEL I:(fitting a normal line in the four regressors)

        regressors = pd.DataFrame()
        for i in range(1, 5):
            regressors['x' + str(i)] = data['x' + str(i)]
        regressors = sm.add_constant(regressors)
        model1 = sm.OLS(outcome, regressors)
        result1 = model1.fit()
        RSS_accumulator['result 1'] = result1.ssr

        # HYPOTHESISED MODEL II:
        # another model, simple.
        # y_i = alpha + b1*x1_i + x2*x3*b2 + epsilon_i
        regressors['x2'] = regressors['x2']*regressors['x3']
        regressors = regressors.drop(columns = ['x3'])


        result2 = sm.OLS(outcome, regressors).fit()
        RSS_accumulator['result 2']= result2.ssr

        # HYPOTHESISED MODEL III:
        # another simple model.
        # y_i = alpha + b1*x1*x2 + b2*x2*x3 + b4*x2*x4 + epsilon_i

        del(regressors)
        regressors = pd.DataFrame()
        for i in range(1, 5):
            regressors['x' + str(i)] = data['x' + str(i)]
        regressors = sm.add_constant(regressors)
        regressors['x1'] = regressors['x1']*regressors['x2']
        regressors['x3'] = regressors['x3']*regressors['x2']
        regressors['x4'] = regressors['x4']*regressors['x2']
        regressors = regressors.drop(columns = ['x2'])

        result3 = sm.OLS(outcome, regressors).fit()
        RSS_accumulator['result 3']= result3.ssr

        # HYPOTHESISED MODEL IV: choosing all the second order interaction terms
        list1 = list(combinations([i for i in range(1, 5)], 2))

        # Now generating a new list of regressors,

        regressors = pd.DataFrame()
        for i in range(len(list1)):
            regressors['x' + str(i)] = data['x' + str(list1[i][0])]*data['x' + str(list1[i][1])]
        for i in range(5, 9):
            regressors['x' + str(i)] = data['x' + str(i - 4)]

        # Running OLS of the regressors on the outcome
        result4 = sm.OLS(outcome, regressors).fit()
        RSS_accumulator['result 4']= result4.ssr

        # HYPOTHESISED MODEL V: choosing all the first, second and third order terms
        list1 = list(combinations([i for i in range(1, 5)], 3))
        for i in range(9, 9 + len(list1)):
            regressors['x' + str(i)] = data['x' + str(list1[i-9][0])]*data['x' + str(list1[i-9][1])]*data['x' + str(list1[i-9][2])]

        result5 = sm.OLS(outcome, regressors).fit()
        #RSS_accumulator['result 5'] = result5.ssr

        #output_object = summary_col([original, result1, result2, result3], stars = True)
        #print(output_object)

        # HYPOTHESISED MODEL VI: Cox proportional hazard model


        # Note that across all these regressions, the values that the mentioned regressors take are different; for the purposes of 
        # brevity, they have not been mentioned in full. 
        #print(results.params)
        #print('The residual sum of squares for this model is: ', results.ssr)
        #print(result.summary(slim = True).as_latex())
        #for i in RSS_accumulator:
        #    print(i, RSS_accumulator[i])
        solution_1 = miniMax(RSS_accumulator)


        test = RSS_accumulator.copy()
        del test['Original']

        algo1_result = miniMax(test)
        algo2_result = minimax_regret(test)
        #print(algo1_result)
        #print(algo2_result)


        loss_1.append(RSS_accumulator[algo1_result] - RSS_accumulator['Original'])
        loss_2.append(RSS_accumulator[algo2_result] - RSS_accumulator['Original'])
        performance = p_val(loss_1[index], loss_2[index], RSS_accumulator, algo1_result, algo2_result)
        #print('Performance for algo. 1:', performance[0])
        #print('Performance for algo. 2:', performance[1])
        p_1 += performance[0]
        p_2 += performance[1]
        index += 1
    return (loss_1, loss_2, p_1/50, p_2/50)

## Data Plotting and Visualization

In [41]:
# I could have done the following exercise with a loop, but I am bored right now.

figure, axis = plt.subplots(nrows = 2, ncols = 2, layout = 'constrained')
x_axis = [i for i in range(100, 5000, 100)]
loss_1, loss_2, a1, a2 = combinator(5000)
p1,=axis[0, 0].plot(x_axis, loss_1)
axis[0, 0].set_title('Losses in Algorithms')
axis[0, 0].set_xlabel('Sample Size')
axis[0, 0].set_ylabel('Loss')
axis[0, 0].plot(x_axis, loss_2)
print(a1, a2)
#plt.show()

loss_1, loss_2, a1, a2  = combinator(5000)
p2,=axis[0, 1].plot(x_axis, loss_1)
axis[0, 1].set_title('Losses in Algorithms')
axis[0, 1].set_xlabel('Sample Size')
axis[0, 1].set_ylabel('Loss')
axis[0, 1].plot(x_axis, loss_2)
print(a1, a2)

loss_1, loss_2, a1, a2 = combinator(5000)
p3,=axis[1, 0].plot(x_axis, loss_1)
axis[1, 0].set_title('Losses in Algorithms')
axis[1, 0].set_xlabel('Sample Size')
axis[1, 0].set_ylabel('Loss')
axis[1, 0].plot(x_axis, loss_2)
print(a1, a2)

loss_1, loss_2, a1, a2 = combinator(5000)
p4,=axis[1, 1].plot(x_axis, loss_1)
axis[1, 1].set_title('Losses in Algorithms')
axis[1, 1].set_xlabel('Sample Size')
axis[1, 1].set_ylabel('Loss')
p5,=axis[1, 1].plot(x_axis, loss_2)
print(a1, a2)

axis[1, 1].legend((p4, p5), ('Algo 1', 'Algo 2'), loc = 'upper right', shadow = True)
figure.savefig('graph_performance.png')
plt.close(figure)

0.0 0.76
0.0 0.77
0.0 0.77
0.0 0.755
